In [1]:
import stim
import time
import random
import numpy as np

for d in range(3, 52, 2):
    circuit = stim.Circuit()

    distance = d
    rounds = 1

    n = 2 * pow(distance, 2) + 1

    grid_size = distance + 1

    def q(i, j):
        return i * grid_size + j

    for i in range(distance):
        for j in range(distance):
            if (i + j) % 2 == 1:
                ancilla = q(i, j)
                data_neighbors = [q(i+1, j), q(i, j+1), q(i-1, j), q(i, j-1)]
                data_neighbors = [n for n in data_neighbors if 0 <= n < grid_size**2]

                circuit.append("H", [ancilla])
                for data in data_neighbors:
                    circuit.append("CX", [ancilla, data])
                circuit.append("H", [ancilla]) 
                circuit.append("M", [ancilla]) 

    for i in range(distance):
        for j in range(distance):
            if (i + j) % 2 == 0: 
                ancilla = q(i, j)
                data_neighbors = [q(i+1, j), q(i, j+1), q(i-1, j), q(i, j-1)]
                data_neighbors = [n for n in data_neighbors if 0 <= n < grid_size**2]

                for data in data_neighbors:
                    circuit.append("CX", [data, ancilla])
                circuit.append("M", [ancilla])

    #Inject an X error on a data qubit
    #circuit.append("X_ERROR(0.1)", [q(1, 1)]) 

    # samples = circuit.sample(num_samples=5)
    # print("\n Measurement Samples:")
    # print(len(circuit))

    start = time.perf_counter()

    simulator = stim.TableauSimulator()
    simulator.do(circuit)

    end = time.perf_counter()


    filename = "/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/surface_code_test/stim_"+ (str)(distance) + ".txt"
    with open(filename, "w") as file:
        file.write("stim\n")
        file.write((str)(end-start)+"\n")
        file.write((str)(distance)+"\n")
        file.write((str)(rounds)+"\n")
        file.write((str)(n)+"\n")

In [10]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import time

def create_surface_code_circuit(distance, rounds):
    grid_size = distance + 1
    circuit = QuantumCircuit(grid_size ** 2, grid_size ** 2)
    
    def q(i, j):
        return i * grid_size + j
    
    for _ in range(rounds):
        for i in range(distance):
            for j in range(distance):
                if (i + j) % 2 == 1:
                    ancilla = q(i, j)
                    data_neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
                    data_neighbors = [q(x, y) for x, y in data_neighbors if 0 <= x < grid_size and 0 <= y < grid_size]
                    
                    circuit.h(ancilla)
                    for data in data_neighbors:
                        circuit.cx(ancilla, data)
                    circuit.h(ancilla)
                    circuit.measure(ancilla, ancilla)
        
        for i in range(distance):
            for j in range(distance):
                if (i + j) % 2 == 0:
                    ancilla = q(i, j)
                    data_neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
                    data_neighbors = [q(x, y) for x, y in data_neighbors if 0 <= x < grid_size and 0 <= y < grid_size]
                    
                    for data in data_neighbors:
                        circuit.cx(data, ancilla)
                    circuit.measure(ancilla, ancilla)
    
    return circuit

for d in range(1, 15, 2):
    distance = d
    rounds = 1000
    circuit = create_surface_code_circuit(distance, rounds)
    
    start = time.perf_counter()
    simulator = AerSimulator(method="stabilizer")
    transpiled_circuit = transpile(circuit, simulator)
    simulator.run(transpiled_circuit).result()
    end = time.perf_counter()
    
    filename = f"/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/surface_code_test/qiskit_{distance}.txt"
    with open(filename, "w") as file:
        file.write("qiskit\n")
        file.write(f"{end-start}\n")
        file.write(f"{distance}\n")
        file.write(f"{rounds}\n")
        file.write(f"{(2 * distance**2) + 1}\n")


In [11]:
import cirq
import time
import numpy as np

def create_surface_code_circuit(distance, rounds):
    grid_size = distance + 1
    qubits = [[cirq.GridQubit(i, j) for j in range(grid_size)] for i in range(grid_size)]
    circuit = cirq.Circuit()
    
    def q(i, j):
        return qubits[i][j]
    
    for _ in range(rounds):
        for i in range(distance):
            for j in range(distance):
                if (i + j) % 2 == 1:
                    ancilla = q(i, j)
                    data_neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
                    data_neighbors = [q(x, y) for x, y in data_neighbors if 0 <= x < grid_size and 0 <= y < grid_size]
                    
                    circuit.append(cirq.H(ancilla))
                    for data in data_neighbors:
                        circuit.append(cirq.CX(ancilla, data))
                    circuit.append(cirq.H(ancilla))
                    circuit.append(cirq.measure(ancilla))
        
        for i in range(distance):
            for j in range(distance):
                if (i + j) % 2 == 0:
                    ancilla = q(i, j)
                    data_neighbors = [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
                    data_neighbors = [q(x, y) for x, y in data_neighbors if 0 <= x < grid_size and 0 <= y < grid_size]
                    
                    for data in data_neighbors:
                        circuit.append(cirq.CX(data, ancilla))
                    circuit.append(cirq.measure(ancilla))
    
    return circuit

for d in range(1, 15, 2):
    distance = d
    rounds = 1000
    circuit = create_surface_code_circuit(distance, rounds)
    
    start = time.perf_counter()
    simulator = cirq.CliffordSimulator()
    simulator.run(circuit)
    end = time.perf_counter()
    
    filename = f"/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/surface_code_test/cirq_{distance}.txt"
    with open(filename, "w") as file:
        file.write("cirq\n")
        file.write(f"{end-start}\n")
        file.write(f"{distance}\n")
        file.write(f"{rounds}\n")
        file.write(f"{(2 * distance**2) + 1}\n")